## Convolutional Neural Network with Flux Framework

#### Muhammad Gaffar
----------------------------------

It is known that to get more generalized model for complex data, we need more bigger parameters and layers on our deep neural network. But, to do that, we also need bigger computational resources. To tackle this problem, Yann LeCunn proposed in 1988 a convolution network. A network that can compress the data, and also figuring out the correlation in subspace of the data by gradient descent. 

Here's a simple architecture of CNN

![arch](fig/CNN_arch.png)

*image is retrieved from introduction to cnn from class cs231a stanford*

what is CNN do is that there are filter layer that have less size than the data, that the filter with weight inside it will slide across the data and calculte the dot product of it. The output data is can be seen as compressed data.

![filter](fig/ConvFilter.png)


There are also a padding and stride, but that is not central on CNN. They are just additional tools to get the compressed data size right. The final volume size of the data is

$$
W_2 = \frac{W_1 - F + 2P}{S} + 1
$$

where $W_1$ is the previous or input weight, $F$ is size of the filter, $P$ is size of zero padding. For height same as weight. and for number of channels

$$
D = K
$$

where $K$ is number of filter

In this tutorial, I will shown the CNN easily using Flux framework.

We load the FLUX and the our toy data (MNIST)

In [ ]:
using Flux
using Flux.Data.MNIST

# Now we load MNIST data from Flux Library
imgs = MNIST.images(:train);
labels = MNIST.labels(:train);

#display the MNIST image from given index
index = 5491
display(imgs[index])
println("label = ", labels[index])

In [ ]:
#we are gonna need some tools.
using Flux: onehotbatch, onecold
using Base.Iterators: partition

In [ ]:
"""
make minibatch data 
    X = images data
    Y = labels data
    idxs = vector range of index
"""
function make_minibatch(X, Y, idxs)
    #initialize array, with dimension (Height,Width,Channel,BatchIndex) WHCN order
    #this is used, so conv filter in flux can be used
    X_batch = Array{Float32}(undef, size(X[1])..., 1, length(idxs))
    #fill array data to minibatch
    for i in 1:length(idxs)
        X_batch[:, :, :, i] = Float32.(X[idxs[i]])
    end
    Y_batch = onehotbatch(Y[idxs], 0:9)
    return (X_batch, Y_batch)
end

# let's see first what onehotbatch data do
onehotbatch(labels[1:10],0:9)

we are gonna use partition function that we get from base.iterators, let's see what partition do

In [ ]:
#we use partition function as list of index that contain every batch
#minibatch index
batch_size = 256
mb_idxs = partition(1:length(imgs), batch_size);

#let's see the minibatch indexes
collect(mb_idxs)

In [ ]:
#now we make the dataset by from minibatch index
train_set = [make_minibatch(imgs, labels, i) for i in mb_idxs];

#load test data and convert to one batch only
test_imgs = MNIST.images(:test)
test_labels = MNIST.labels(:test)
test_set = make_minibatch(test_imgs, test_labels, 1:length(test_imgs));

Flux gave us a simple way to build model, using chain function

In [ ]:
#Now we build our very simple model, consist of three conv layer, and one dense layer
model = Chain(
    # First convolution, operating upon a 28x28 image
    Conv((3, 3), 1=>16, pad=(1,1), relu),
    MaxPool((2,2)),

    # Second convolution, operating upon a 14x14 image
    Conv((3, 3), 16=>32, pad=(1,1), relu),
    MaxPool((2,2)),
    
    # Third convolution, operating upon a 7x7 image
    Conv((3, 3), 32=>32, pad=(1,1), relu),
    MaxPool((2,2)),

    # Flatten the data , 3x3x32 = 288
    # which is where we get the 288 in the `Dense` layer below:
    x -> reshape(x, :, size(x, 4)),
    Dense(288, 10),

    # Finally, softmax to get nice probabilities
    softmax,
)

#compile the model
model(train_set[1][1]);

#now we define our loss function using crossentropy
using Flux: crossentropy
loss(x,y) = crossentropy(model(x),y)

#model accuracy
using Statistics
accuracy(x,y) = mean(onecold(model(x)) .== onecold(y));

#and the optimizer using gradient descent
η = 0.005 #learning rate
opt = Descent(η);

we train our model using function within flux, `Flux.train!`.

In [ ]:
@time Flux.train!(loss,params(model),train_set,opt)

In [ ]:
for epoch in 1:15
    # Train for a single epoch
    Flux.train!(loss, params(model), train_set, opt)

    # Calculate accuracy:
    acc = accuracy(test_set...)
    println("Epoch $epoch ==> Accuracy = $acc")
end

even though for full batch train require much more time, we see that for minimum, give epoch high accuracy result. This indeed will be useful if we train in more complex data, thus ConvNets give more efficiency.